In [1]:
# 소속 선수 수 상위 5개국
# England Germany Spain Argentina France
# 선수 사진으로 국적 분류

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import os
from PIL import ImageOps
from PIL import Image

import argparse
import sys

In [2]:
# referenced image_to_str in binary_cnn
def image_to_str(path, one_hot_label):
    with open(path, "rb") as f:
        with Image.open(f) as img:
            arr = np.array(img)[:,:,:3]
            return ",".join(map(str, arr.flat)) + "," + ",".join(map(str, one_hot_label))
            

In [3]:
path = "C:/Users/ssm74/Projects/WAYF/Data/valid_pictures/England/54050.png"
image_to_str(path, [1, 0, 0, 0, 0])

'0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71,65,59,37,28,

In [4]:
# England Germany Spain Argentina France
path = "C:/Users/ssm74/Projects/WAYF/Data/valid_pictures"
name_one_hot = [ ("England", (1, 0, 0, 0, 0)), 
                 ("Germany", (0, 1, 0, 0, 0)),
                 ("Spain", (0, 0, 1, 0, 0)),
                 ("Argentina", (0, 0, 0, 1, 0)),
                 ("France", (0, 0, 0, 0, 1))
               ]

with open("5_cnn_data.csv", "w") as f:
    for name, one_hot in name_one_hot:
        directory_path = path + "/" + name
        for id in os.listdir(directory_path):
            img_path = directory_path + "/" + id
            f.write(image_to_str(img_path, one_hot) + "\n")

In [5]:
arr = []

with open("./5_cnn_data.csv", "r") as f:
    for line in f:
        arr.append(list(map(int, line.split(","))))
        
arr = np.array(arr)
arr.shape

(4806, 6917)

In [10]:
def weight_variable(shape):
    '''
    파라미터(가중치 텐서)의 초깃값을 정하는 함수
    표준편차가 0.1인 절단정규분포 이용
    '''
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial, name="weights")

def bias_variable(shape):
    '''
    편향값의 초깃값을 정하는 함수
    '''
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name="bias")

In [11]:
def conv2d(x, w):
    '''
    합성곱을 구하는 함수
    입력값 x를 필터 텐서 w를 이용하여 변형
    stride = 1, padding = SAME
    변형된 이미지는 입력 이미지와 높이, 너비가 같음
    '''    
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="SAME", name="conv")

def max_pool_2x2(x):
    '''
    2x2 윈도우 안의 최댓값을 뽑아 새로운 텐서 생성
    stride가 2이기 때문에 출력 텐서의 높이와 너비는 입력 텐서의 1/2가 됨
    '''    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME", name="pool")


In [12]:
def main(_):
    '''
    모델 학습과 결과 출력
    '''
    
    x = tf.placeholder(tf.float32, [None, 48, 48, 3], name="x")
    y = tf.placeholder(tf.float32, [None, 5], name="y")
    keep_prob = tf.placeholder(tf.float32)
    
    train_size = 3364
    image_size = 6912
    
    with tf.name_scope("first_layer"):
        w_conv1 = weight_variable([3, 3, 3, 32])
        b_conv1 = bias_variable([32])
        
        h_conv1 = tf.nn.relu(conv2d(x, w_conv1) + b_conv1, name="h_conv1")
        # h_conv1 shape = [?, 48, 48, 32]
        
        h_pool1 = max_pool_2x2(h_conv1)
        # h_pool1 shape = [?, 24, 24, 32]
        
    with tf.name_scope("second_layer"):
        w_conv2 = weight_variable([3, 3, 32, 64])
        b_conv2 = bias_variable([64])
        
        h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2, name="h_conv2")
        # h_conv2 shape = [?. 24, 24, 64]
        
        h_pool2 = max_pool_2x2(h_conv2)
        # h_pool2 shape = [?, 12, 12, 64]
    
    with tf.name_scope("third_layer"):
        w_conv3 = weight_variable([3, 3, 64, 128])
        b_conv3 = bias_variable([128])
        
        h_conv3 = tf.nn.relu(conv2d(h_pool2, w_conv3) + b_conv3, name="h_conv3")
        # h_conv3 shape = [?. 12, 12, 128]
        
        h_pool3 = max_pool_2x2(h_conv3)
        # h_pool2 shape = [?, 6, 6, 128]
    
    with tf.name_scope("first_fc"):
        w_fc1 = weight_variable([6*6*128, 1024])
        b_fc1 = bias_variable([1024])
        
        h_pool3_flat = tf.reshape(h_pool3, [-1, 6*6*128])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, w_fc1) + b_fc1, name="h_fc1")
        
    with tf.name_scope("drop_out"):
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        
    with tf.name_scope("second_fc"):
        w_fc2 = weight_variable([1024, 5])
        b_fc2 = bias_variable([5])
        
    y_conv = tf.matmul(h_fc1_drop, w_fc2) + b_fc2
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y),
                                name="croess_entropy")
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for i in range(100):
            np.random.shuffle(arr)
            trX = arr[:train_size, :image_size].reshape((-1, 48, 48, 3))
            trY = arr[:train_size, image_size:]
            
            sess.run(train_step, feed_dict={x: trX, y: trY, keep_prob:0.5})
            
            teX = arr[train_size: , :image_size].reshape((-1, 48, 48, 3))
            teY = arr[train_size: , image_size:]
            
            # 평가 시에는 완성된 모델을 사용하기 위해
            # dropout을 하지 않으므로 keep_prob = 1
            train_accuracy = accuracy.eval(feed_dict={x: teX, y:teY, keep_prob:1.0})
            print("step: %d, train_accuracy: %g" % (i, train_accuracy))
            
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)   

step: 0, train_accuracy: 0.317614
step: 1, train_accuracy: 0.308599
step: 2, train_accuracy: 0.300277
step: 3, train_accuracy: 0.29681
step: 4, train_accuracy: 0.296117
step: 5, train_accuracy: 0.29681
step: 6, train_accuracy: 0.307212
step: 7, train_accuracy: 0.298197
step: 8, train_accuracy: 0.301664
step: 9, train_accuracy: 0.319001
step: 10, train_accuracy: 0.304438
step: 11, train_accuracy: 0.332178
step: 12, train_accuracy: 0.334951
step: 13, train_accuracy: 0.32663
step: 14, train_accuracy: 0.334258
step: 15, train_accuracy: 0.327323
step: 16, train_accuracy: 0.352288
step: 17, train_accuracy: 0.365465
step: 18, train_accuracy: 0.402913
step: 19, train_accuracy: 0.395284
step: 20, train_accuracy: 0.375173
step: 21, train_accuracy: 0.409847
step: 22, train_accuracy: 0.399445
step: 23, train_accuracy: 0.385576
step: 24, train_accuracy: 0.417476
step: 25, train_accuracy: 0.409154
step: 26, train_accuracy: 0.399445
step: 27, train_accuracy: 0.414008
step: 28, train_accuracy: 0.39597

SystemExit: 